#### For tax

In [1]:
tax_tax_raw = ['data_2000_tax_5feats_tax_k1_gpt4.json', 'data_2000_tax_10feats_tax_k1_gpt4.json']


In [23]:
import json

directory = 'outputs/'
with open(directory + 'data_2000_tax_15feats_tax_k1_gpt4.json', 'r') as file:
    generate = json.load(file)

eval(generate[0]['output'])

[['Mystery', 'Thriller'],
 ['Crime', 'Justice', 'Family'],
 'Adults',
 'Contemporary',
 'Popular',
 'English',
 'Paperback',
 'Novel',
 '2001-2010',
 'American',
 'Male',
 '301-400',
 'Intermediate',
 'None',
 'Unknown']

In [24]:
n_error = 0
error_index = []
for index, entry in enumerate(generate):
    try:
        # eval(entry['output'].replace('\"\n', '\n').replace('):', ')":')) # for other tax files
        # eval(entry['output'].replace('```', '')) # for 'data_2000_tax_20feats_name_k1_gpt4.json'
        eval(entry['output'])
    except:
        # n_error += 1
        # print(index)
        # error_index.append(index)
        # print(entry['output'])
        try:
            eval(entry['output'].replace(']]', ']'))
            # eval('{'+entry['output'].replace('```', '')+'}')
        except:
            n_error += 1
            print(index)
            error_index.append(index)
            print(entry['output'])

n_error

0

In [27]:
for index, entry in enumerate(generate):
    try:
        eval(entry['output'])
    except:
        entry['output'] = entry['output'].replace(']]', ']')

In [13]:
def flatten_nested_list(nested_list):
    return [item for sublist in nested_list for item in (sublist if isinstance(sublist, list) else [sublist])]

In [28]:
for entry in generate:
    output = eval(entry['output'])
    # flatten the output
    # processed_output = flatten_nested_list(output)
    processed_output = flatten_nested_list(output) # same as flatten_book_dict(nested_list) below
    print(processed_output)

['Mystery', 'Thriller', 'Crime', 'Justice', 'Family', 'Adults', 'Contemporary', 'Popular', 'English', 'Paperback', 'Novel', '2001-2010', 'American', 'Male', '301-400', 'Intermediate', 'None', 'Unknown']
['Historical Fiction', 'Romance', 'Love', 'Adventure', 'Family', 'Adults', 'Historical', 'Popular', 'English', 'Paperback', 'Novel', '1951-2000', 'American', 'Female', '301-400', 'Intermediate', 'None', 'Unknown']
['Thriller', 'Fantasy', 'Crime', 'Magic', 'Justice', 'Adults', 'Contemporary', 'Popular', 'English', 'Paperback', 'Novel', '2001-2010', 'American', 'Male', '301-400', 'Intermediate', 'None', 'Unknown']
['Science Fiction', 'Thriller', 'Technology', 'Survival', 'Adventure', 'Adults', 'Contemporary', 'Popular', 'English', 'Paperback', 'Novel', '2001-2010', 'American', 'Male', '401-500', 'Intermediate', 'None', 'Unknown']
['Thriller', 'Mystery', 'Romance', 'Crime', 'Love', 'Family', 'Adults', 'Contemporary', 'Popular', 'English', 'Paperback', 'Novel', '2001-2010', 'American', 'Fem

In [32]:
# for rec with name

import json
from rouge_score import rouge_scorer

# Calculate the Rouge-L score between the target book and all candidate books
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True) # Use Rouge-L scorer to calculate the similarity between the target book and the candidate books

def list_dimensions(lst):
    """递归函数，用以确定列表的维度。"""
    if not isinstance(lst, list):
        return 0
    if len(lst) == 0:
        return 1
    return 1 + max(list_dimensions(item) for item in lst)

def calculate_tax_similarity(target_taxonomy, book_taxonomy):
    """Calculate the similarity as the count of common words."""
    return len(set(target_taxonomy) & set(book_taxonomy))/len(set(target_taxonomy))

def find_top_k_similar_books_nametax(target_book, book_dict, k, alpha=1, beta=1):
    """Find the top-K similar books based on taxonomy similarity."""
    # target_book = [book_name, book_taxonomy]
    # book_taxonomy = [feature1, feature2, ...]
    # Calculate similarity scores for each book in the dictionary
    target_book_name = target_book[0]
    target_book_taxonomy = target_book[1]  # a list of features values
    similarity_scores = []
    for book_name, book_taxonomy in book_dict.items():
        name_score = scorer.score(target_book_name, book_name)['rougeL'].fmeasure
        tax_score = calculate_tax_similarity(target_book_taxonomy, book_taxonomy)
        similarity = alpha * name_score + beta * tax_score
        # similarity = calculate_tax_similarity(target_book_taxonomy, book_taxonomy)
        similarity_scores.append((book_name, similarity))

    # Sort the books based on similarity score in descending order and select the top-K
    top_k_similar_books = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[:k]

    books_list = [book for book, _ in top_k_similar_books]
    scores_list = [score for _, score in top_k_similar_books]

    return books_list, scores_list

def find_top_k_similar_books_tax(target_book, book_dict, k):
    """Find the top-K similar books based on taxonomy similarity."""
    # target_book = [book_taxonomy]
    # book_taxonomy = [feature1, feature2, ...]
    # Calculate similarity scores for each book in the dictionary
    target_book_taxonomy = target_book  # a list of features values
    similarity_scores = []
    for book_name, book_taxonomy in book_dict.items():
        similarity = calculate_tax_similarity(target_book_taxonomy, book_taxonomy)
        similarity_scores.append((book_name, similarity))

    # Sort the books based on similarity score in descending order and select the top-K
    top_k_similar_books = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[:k]

    books_list = [book for book, _ in top_k_similar_books]
    scores_list = [score for _, score in top_k_similar_books]

    return books_list, scores_list

def process_generate_name(generate_data):
    for entry in generate_data:
        try:
            # output = eval(entry['output'].replace('\"\n', '\n').replace('):', ')":'))
            output = eval(entry['output'])
        except:
            print(entry['output'])

        if isinstance(output, dict):
            processed_output = list(output.keys())
            rec_tax = next(iter(output.values()))
            if list_dimensions(rec_tax) == 1:
                processed_output.append(rec_tax)
            elif list_dimensions(rec_tax) == 2:
                flatten_rec_tax = [item for sublist in rec_tax for item in sublist]
                processed_output.append(flatten_rec_tax)
        entry['top1_book'] = processed_output
    return generate_data

def process_generate_tax(generate_data):
    for entry in generate_data:
        try:
            output = eval(entry['output'])
        except:
            output = eval(entry['output'].replace(']]', ']'))

        processed_output = flatten_nested_list(output)
        entry['top1_book'] = processed_output
    return generate_data

def flatten_book_dict(nested_list):
    """递归地将嵌套列表平铺为一维列表"""
    flat_list = []
    for item in nested_list:
        if isinstance(item, list):
            flat_list.extend(flatten_book_dict(item))
        else:
            flat_list.append(item)
    return flat_list

def flatten_books_dict(books_dict):
    for book, book_tax in books_dict.items():
        # try:
        #     temp_dict = eval(book_tax)
        # except:
        #     print(book)
        # temp_dict = eval(book_tax)
        temp_dict = book_tax
        value_list = []
        for value in temp_dict.values():
            value_list.append(value)
        value_list = flatten_book_dict(value_list)
        books_dict[book] = value_list
    
    return books_dict


In [35]:
from tqdm import tqdm
import json

output_dir = 'outputs/'

tax_tax_raw = ['data_2000_tax_5feats_tax_k1_gpt4.json', 
               'data_2000_tax_10feats_tax_k1_gpt4.json',
               'data_2000_tax_15feats_tax_k1_gpt4.json']


dict_dir = 'processed_data/'
book_dicts = ['book_taxonomy_5feats.json', 
                'book_taxonomy_10feats.json',
                'book_taxonomy_15feats.json']


generate_file_name = 'data_2000_tax_5feats_tax_k1_gpt4.json'
with open(output_dir + generate_file_name, 'r') as file:
    generate = json.load(file)

with open(dict_dir + "book_taxonomy_5feats.json", "r") as file:
    books_dict = json.load(file)

books_dict_flatten = flatten_books_dict(books_dict)
generate_processed = process_generate_tax(generate)


for entry in tqdm(generate_processed):
    top1_rec_book = entry['top1_book']
    top10_rec_books, top10_scores = find_top_k_similar_books_tax(top1_rec_book, books_dict_flatten, k=10)
    entry['top10_similar_for_top1'] = top10_rec_books
    entry['top10_scores'] = top10_scores


with open(output_dir + 'data_2000_tax_5feats_tax_k1_gpt4_processed.json', 'w') as file:
    json.dump(generate_processed, file, indent=2)

100%|██████████| 2000/2000 [00:09<00:00, 211.64it/s]


In [38]:
def calculate_dcg(scores):
    """Calculate DCG for the given relevance scores."""
    import numpy as np
    return np.sum([
        (2**score - 1) / np.log2(idx + 2) for idx, score in enumerate(scores)
    ])

def calculate_ndcg_at_k(true_book_name, books_list, k):
    """Calculate NDCG@k for a given true book name and a list of book names."""
    # Assign relevance scores
    relevance_scores = [1 if name == true_book_name else 0 for name in books_list]
    
    # Calculate DCG@k
    dcg_at_k = calculate_dcg(relevance_scores[:k])
    
    # Calculate IDCG@k (ideal case where the true book is at the top)
    idcg_at_k = calculate_dcg([1] + [0]*(k-1))
    
    # Calculate NDCG@k
    ndcg_at_k = dcg_at_k / idcg_at_k
    return ndcg_at_k

def calc_metrics_at_k(output, k=10):

    recall_count = 0  # Count of correct recommendations
    ndcg_score = 0  # Accumulated NDCG score

    for entry in output:
        recommended_books = entry['top10_similar_for_top1']  # List of recommended book names
        true_book_name = entry['true_book'].strip('"')  # True book name

        if true_book_name in recommended_books[:k]:
            recall_count += 1
        
        # Calculate NDCG@K
        ndcg_score += calculate_ndcg_at_k(true_book_name, recommended_books, k)

    recall_at_k = recall_count / len(output)
    ndcg_at_k = ndcg_score / len(output)
    
    return recall_at_k, ndcg_at_k

In [39]:
import json

directory = 'outputs/'

gpt4 = [directory + 'data_2000_dir_k1_gpt4_processed.json',
         directory + 'data_2000_tax_5feats_tax_k1_gpt4_processed.json',
         directory + 'data_2000_tax_10feats_tax_k1_gpt4_processed.json',
         directory + 'data_2000_tax_15feats_tax_k1_gpt4_processed.json']

for k in [1, 5, 10]:
    print(f"@{k}")
    for file in gpt4:
        print(file.strip('_processed.json').split('2000_')[1])
        with open(file, 'r') as file:
            results = json.load(file)
        # recall_at_k, ndcg_at_k = calc_metrics_at_k(results, book_dict, k=k)
        recall_at_k, ndcg_at_k = calc_metrics_at_k(results, k=k)
        print(f"Recall@{k}: {recall_at_k:.4f}, NDCG@{k}: {ndcg_at_k:.4f}")
    print('------------------------------------------------------')


@1
dir_k1_gpt4
Recall@1: 0.0000, NDCG@1: 0.0000
tax_5feats_tax_k1_gpt4
Recall@1: 0.0030, NDCG@1: 0.0030
tax_10feats_tax_k1_gpt4
Recall@1: 0.0070, NDCG@1: 0.0070
tax_15feats_tax_k1_gpt4
Recall@1: 0.0040, NDCG@1: 0.0040
------------------------------------------------------
@5
dir_k1_gpt4
Recall@5: 0.0015, NDCG@5: 0.0006
tax_5feats_tax_k1_gpt4
Recall@5: 0.0065, NDCG@5: 0.0046
tax_10feats_tax_k1_gpt4
Recall@5: 0.0150, NDCG@5: 0.0109
tax_15feats_tax_k1_gpt4
Recall@5: 0.0160, NDCG@5: 0.0098
------------------------------------------------------
@10
dir_k1_gpt4
Recall@10: 0.0025, NDCG@10: 0.0010
tax_5feats_tax_k1_gpt4
Recall@10: 0.0115, NDCG@10: 0.0062
tax_10feats_tax_k1_gpt4
Recall@10: 0.0240, NDCG@10: 0.0138
tax_15feats_tax_k1_gpt4
Recall@10: 0.0265, NDCG@10: 0.0132
------------------------------------------------------
